In [4]:
# DataFrame
import pandas as pd
pd.set_option('display.max_colwidth',None)
from tqdm import tqdm_notebook, tnrange

# Matplot
import matplotlib.pyplot as plt
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM,Input
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping , CSVLogger
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.layers import LSTM, Dense, Embedding, Bidirectional,GlobalMaxPool1D, Lambda
from keras.layers import TimeDistributed
from keras.layers import *
from keras.optimizers import Adam,Nadam

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim
# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

#spacy
import wordninja
from nltk.tokenize import TweetTokenizer
stop_words = set(stopwords.words('english'))
tknzr = TweetTokenizer()


from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

In [5]:
train=pd.read_csv('./data/train.csv')
test=pd.read_csv('./data/test.csv')

In [6]:
train.head()
train=train.sample(frac=1)

In [7]:
train.label.value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [8]:
model = ClassificationModel('roberta', 'roberta-base',use_cuda=False) # You can set class weights by using the optional weight argument


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [ ]:
#train the model
model.train_model(train_df)

/Users/subir/pythonenv/default/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:251: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


Running loss: 0.405847

/Users/subir/pythonenv/default/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.516560

/Users/subir/pythonenv/default/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 0.463969

In [143]:
Submission = pd.DataFrame()
Submission['id']=df_test['id']
Submission['label']=sentiment

In [135]:
Submission.to_csv('Submission_BiLSTM_Fasttext.csv',index=False)